In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt


# set mpl font sizing
SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc("font", size=BIGGER_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
font_family = "serif"

# set seaborn styling
sns.set_style("whitegrid")
n_colors = 9
cms_color = {
    color: sns.color_palette(color, n_colors)
    for color in ["Blues", "Greens", "Reds", "Oranges", "Purples", "Greys"]
}

# make figure saving dir
fig_dir = "figures"
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
from avapi.carla import CarlaScenesManager


data_dir = "/data/shared/CARLA/multi-agent-aerial"
# data_dir = "/data/shared/CARLA/multi-agent-aerial-dense"
CSM = CarlaScenesManager(data_dir=data_dir)
CDM = CSM.get_scene_dataset_by_index(0)

In [ ]:
from avstack.geometry import GlobalOrigin3D
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np


def plot_scene(
    agents,
    fovs,
    objs,
    fig_title,
    extent=[-147, 140, -175, 100],
    background_img=None,
):
    # Define colormap
    cmap = plt.get_cmap(
        "tab10", len(agents)
    )  # Changed colormap for better color contrast

    # Create figure and axis
    fig, ax = plt.subplots(
        1, 1, figsize=(8, 8)
    )  # Increased figure size for better visibility

    # Loop over agents to plot each
    for i, agent in enumerate(agents):
        # Assign color based on colormap
        color = cmap(i)

        # add background image
        if background_img is not None:
            ax.imshow(background_img, extent=extent)

        # Plot agent position with larger marker
        ax.scatter(
            x=agent.position.x[0],
            y=agent.position.x[1],
            label=f"Agent {i}",
            color=color,
            marker="o",
            s=100,  # Increased marker size for visibility
            edgecolor="black",  # Added border to the marker for clarity
        )

        # squeeze the patch inside extent
        pass

        # Add FOV rectangle with better transparency and border
        patch = Rectangle(
            xy=fovs[i]["xy"],
            width=fovs[i]["width"],
            height=fovs[i]["height"],
            angle=fovs[i]["angle"],
            rotation_point="center",
            facecolor=color,
            alpha=0.5,  # Adjusted transparency
            edgecolor="black",  # Added edge color for distinction
            linewidth=1,  # Set border width
        )
        ax.add_patch(patch)

    # Plot object positions with larger markers
    objs_pos = np.array([obj.position.x[:2] for obj in objs])
    ax.scatter(
        objs_pos[:, 0],
        objs_pos[:, 1],
        color="blue",
        marker="x",
        s=60,  # Increased marker size for objects
        linewidths=1.5,  # Increased line width of the marker for clarity
    )

    # Adding labels, grid, and legend
    ax.set_title("Agent Coverage in CARLA Town", fontweight="bold")
    ax.grid(
        visible=True, linestyle="--", alpha=0.5
    )  # Modified grid style for less prominence
    ax.set_xticks([])
    ax.set_yticks([])
    ax.legend(loc="upper right", fontsize=10)  # Added legend to distinguish agents

    # Display the plot
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, fig_title))
    plt.show()


def load_agents_objects(idx_frame, n_agents=np.inf):
    # Get agent positions
    frame = CDM.get_frames(sensor="camera-0", agent=0)[idx_frame]
    agents = CDM.get_agents(frame)
    agents = agents[: min(len(agents), n_agents)]
    objs = CDM.get_objects_global(frame=frame, ignore_static_agents=True)

    # compute FOV given sensor parameters
    fovs = []
    for i in range(len(agents)):
        # get the half fov angles for each camera
        calib = CDM.get_calibration(frame=frame, sensor="camera-0", agent=i)
        ref_global = calib.reference.integrate(start_at=GlobalOrigin3D)
        d_az = np.arctan(calib.img_shape[1] / (2 * calib.P[0, 0]))
        d_el = np.arctan(calib.img_shape[0] / (2 * calib.P[1, 1]))
        dx = d_az * ref_global.x[2]
        dy = d_el * ref_global.x[2]

        # store the bottom left corner
        fov = {
            "xy": [-dx + agents[i].position.x[0], -dy + agents[i].position.x[1]],
            "angle": agents[i].attitude.yaw,
            "width": 2 * dx,
            "height": 2 * dy,
        }
        fovs.append(fov)

    return agents, fovs, objs

In [ ]:
from PIL import Image
from avapi.visualize.snapshot import show_image_with_boxes


# load and project the objects
idx_agent = 2
idx_frame = 100
frame = CDM.get_frames(sensor="camera-0", agent=0)[idx_frame]
img = CDM.get_image(frame=frame, sensor="camera-0", agent=idx_agent)
objs_img = CDM.get_objects(frame=frame, sensor="camera-0", agent=idx_agent)
objs_img_2d = [obj.box3d.project_to_2d_bbox(img.calibration) for obj in objs_img]

# show 2D boxes
img_view_2d = show_image_with_boxes(
    img=img, boxes=objs_img_2d, inline=True, return_image=True
)
result = Image.fromarray(img_view_2d)
result.save(
    os.path.join(fig_dir, f"2d_image_agent_{idx_agent:02d}_frame_{frame:03d}.jpeg")
)

# show 3D boxes
img_view_3d = show_image_with_boxes(
    img=img, boxes=objs_img, inline=True, return_image=True
)
result = Image.fromarray(img_view_3d)
result.save(
    os.path.join(fig_dir, f"3d_image_agent_{idx_agent:02d}_frame_{frame:03d}.jpeg")
)

In [ ]:
from PIL import Image
from avapi.visualize.snapshot import show_image_with_boxes


for idx_agent in range(6):
    idx_frame = 100
    frame = CDM.get_frames(sensor="camera-0", agent=0)[idx_frame]
    img = CDM.get_image(frame=frame, sensor="camera-0", agent=idx_agent)
    objs_img = CDM.get_objects(frame=frame, sensor="camera-0", agent=idx_agent)
    img_view = show_image_with_boxes(
        img=img, boxes=objs_img, inline=True, return_image=True
    )

    # save figure
    result = Image.fromarray(img_view)
    result.save(
        os.path.join(fig_dir, f"image_agent_{idx_agent:02d}_frame_{frame:03d}.jpeg")
    )

In [ ]:
for idx_frame in [0, 50]:
    # load agents and objects
    background_img = plt.imread("town10.jpg")
    agents, fovs, objs = load_agents_objects(idx_frame=idx_frame, n_agents=10)

    # plot the agents on the scene
    plot_scene(
        agents=agents,
        fovs=fovs,
        objs=objs,
        background_img=background_img,
        fig_title=f"bev_scene_view_frame_{idx_frame:03d}",
    )